In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" # define GPU id, remove if you want to use all GPUs available
import torch
from tqdm import tqdm
import time
from contextlib import contextmanager
import numpy as np
from medusa.model.modeling_llama_kv import LlamaForCausalLM as KVLlamaForCausalLM
from medusa.model.medusa_model import MedusaModel
from medusa.model.kv_cache import *
from medusa.model.utils import *
#from medusa.model.medusa_choices import *
import transformers
from huggingface_hub import hf_hub_download

/var/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=122


  warn((f'\n\n{"="*80}\n'


[2023-12-24 22:09:59,387] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-12-24 22:10:00.894111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" # define GPU id, remove if you want to use all GPUs available
import torch
from tqdm import tqdm
import time
from contextlib import contextmanager
import numpy as np
from medusa.model.modeling_llama_kv import LlamaForCausalLM as KVLlamaForCausalLM
from medusa.model.medusa_model import MedusaModel
from medusa.model.kv_cache import *
from medusa.model.utils import *
#from medusa.model.medusa_choices import *
import transformers
from huggingface_hub import hf_hub_download

In [5]:
model_name = '../../../../idea5_3gram_4fastlayer_t1_skipbert_teacherstudent_2_medusa_mlp_vicuna-7b-v1.3_medusa_1_lr_0.0001_layers_1/checkpoint-1200/pytorch_model.bin'

In [6]:
model = torch.load(model_name)

In [7]:
model_name2 = '../../../../idea5_3gram_4fastlayer_t1_skipbert_teacherstudent_2_medusa_mlp_vicuna-7b-v1.3_medusa_1_lr_0.0001_layers_1'
model2 = MedusaModel.from_pretrained(
    model_name2,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
   
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


path ../../../../../model/vicuna-7b-v1.3
path:  ../../../../../model/vicuna-7b-v1.3


In [8]:
model2.load_state_dict(model)

<All keys matched successfully>

In [9]:
tokenizer = model2.get_tokenizer()

In [10]:
temperature = 0.
posterior_threshold = 0.09
posterior_alpha = 0.3

In [11]:
prompt = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Hi, could you share a tale about a charming llama that grows Medusa-like hair and starts its own coffee shop? ASSISTANT:"

In [13]:
tokenizer = model2.get_tokenizer()

medusa_choices = torch.tensor([2,5])#([5,5,5,5,5])#4,5
num_heads = len(medusa_choices) - 1
medusa_topk = medusa_choices[1:]

medusa_buffers = generate_medusa_buffers(medusa_choices, device=model.base_model.device)

AttributeError: 'collections.OrderedDict' object has no attribute 'base_model'

In [ ]:
past_key_values, past_key_values_data, current_length_data = initialize_past_key_values(model.base_model)

In [ ]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_buffers,
                    medusa_topk,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                    past_key_values,
                    past_key_values_data, current_length_data
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)

In [ ]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)

In [ ]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)